In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [9]:
#load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
## preprocessing the data
# drop the unnecessary columns
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [11]:
#encode the categorical columns
lable_encoder_gender = LabelEncoder()
data['Gender']= lable_encoder_gender.fit_transform(data['Gender'])

In [12]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [14]:
#onehot encoding the geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geography = OneHotEncoder()
geography_encoded = onehot_encoder_geography.fit_transform(data[['Geography']])
geography_encoded

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [15]:
onehot_encoder_geography.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [19]:
onehot_encoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [23]:
gep_encoded_df = pd.DataFrame(geography_encoded.toarray(),columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
gep_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [24]:
#combine the encoded geography columns with the main dataframe
data = pd.concat([data.drop(['Geography'],axis=1),gep_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [25]:
###save the encoders and scaler
with open('lable_encoder_gender.pkl','wb') as f:
    pickle.dump(lable_encoder_gender,f)

with open('onehot_encoder_geography.pkl','wb') as f:
    pickle.dump(onehot_encoder_geography,f)

In [31]:
##splitting the data into training and testing sets
# exited is my dependent feature
X = data.drop(['Exited'],axis=1)
Y = data['Exited']

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)


In [32]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [33]:
#scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [52]:
scaler.feature_names_in_

array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [53]:
#convert the scaled data to pickle files
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [59]:
(X_train.shape[1],)

(12,)

## ANN implementation

In [67]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [60]:
##Build our ANN model
model = Sequential([
    ##Hidden layer 1 connected with input layer layer
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'), ##hidden layer 2
    Dense(1,activation='sigmoid') ##output layer         
])

In [61]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01) #we will use adam optimizer with learning rate of 0.01
loss = tf.keras.losses.BinaryCrossentropy()
opt
loss


<LossFunctionWrapper(<function binary_crossentropy at 0x3179827a0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [65]:
#complile the model
# model.compile(optimizer='adam',loss='binary_corssentropy',metrics=['accuracy'])

#complile the model
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [70]:
##set up the tensorboard is use to visualize the training process
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [73]:
#set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [74]:
## Trainning the model
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test)
                  ,epochs=100,
                  callbacks=[tensorflow_callback,early_stopping_callback]
                  )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8572 - loss: 0.3434 - val_accuracy: 0.8575 - val_loss: 0.3455
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - accuracy: 0.8580 - loss: 0.3395 - val_accuracy: 0.8565 - val_loss: 0.3676
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step - accuracy: 0.8619 - loss: 0.3376 - val_accuracy: 0.8590 - val_loss: 0.3358
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 949us/step - accuracy: 0.8615 - loss: 0.3329 - val_accuracy: 0.8535 - val_loss: 0.3503
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - accuracy: 0.8654 - loss: 0.3307 - val_accuracy: 0.8675 - val_loss: 0.3403
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.8634 - loss: 0.3309 - val_accuracy: 0.8545 - val_loss: 0.3481
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.8637 - loss: 0.3275 - val_accuracy: 0.8615 - val_loss: 0.3386
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.8661 - loss: 0.3

In [75]:
model.save('ann_model.h5')

In [76]:
#load Tensorboard extension
%load_ext tensorboard

In [78]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 92342), started 0:00:53 ago. (Use '!kill 92342' to kill it.)

In [ ]:
### load the pickle files
